# Assignment 8

#### Question 1 Given two strings s1 and s2, return the lowest ASCII sum of deleted characters to make two strings equal.

Example 1:

Input: s1 = "sea", s2 = "eat"

Output: 231

Explanation: Deleting "s" from "sea" adds the ASCII value of "s" (115) to the sum.

Deleting "t" from "eat" adds 116 to the sum.

At the end, both strings are equal, and 115 + 116 = 231 is the minimum sum possible to achieve this.


In [1]:
def min_delete_sum(s1, s2):
    m, n = len(s1), len(s2)
    
    # Create a 2D table to store the minimum ASCII sum of deleted characters
    dp = [[0] * (n+1) for _ in range(m+1)]
    
    # Fill in the table starting from the last characters of s1 and s2
    for i in range(m-1, -1, -1):
        dp[i][n] = dp[i+1][n] + ord(s1[i])
    for j in range(n-1, -1, -1):
        dp[m][j] = dp[m][j+1] + ord(s2[j])
    
    # Fill in the rest of the table
    for i in range(m-1, -1, -1):
        for j in range(n-1, -1, -1):
            if s1[i] == s2[j]:
                dp[i][j] = dp[i+1][j+1]
            else:
                dp[i][j] = min(dp[i+1][j] + ord(s1[i]), dp[i][j+1] + ord(s2[j]))
    
    return dp[0][0]  # Return the minimum ASCII sum for the whole strings

s1 = "sea"
s2 = "eat"
print(min_delete_sum(s1, s2))

231


#### Question 2 Given a string s containing only three types of characters: '(', ')' and '*', return true if s is valid.

The following rules define a valid string:

Any left parenthesis '(' must have a corresponding right parenthesis ')'.

Any right parenthesis ')' must have a corresponding left parenthesis '('.

Left parenthesis '(' must go before the corresponding right parenthesis ')'.

'*' could be treated as a single right parenthesis ')' or a single left parenthesis '(' or an empty string "".

Example 1:

Input: s = "()"

Output:

True


In [2]:
def check_valid_string(s):
    stk = []
    s_stk = []
    
    for i, char in enumerate(s):
        if char == '(':
            stk.append(i)
        elif char == '*':
            s_stk.append(i)
        else:  # char == ')'
            if stk:
                stk.pop()
            elif s_stk:
                s_stk.pop()
            else:
                return False
    
    while stk and s_stk:
        if stk[-1] > s_stk[-1]:
            return False
        stk.pop()
        s_stk.pop()
    
    return len(stk) == 0

s = "()"
print(check_valid_string(s))


True


#### Question 3 Given two strings word1 and word2, return the minimum number of steps required to make word1 and word2 the same. In one step, you can delete exactly one character in either string.

Example 1:

Input: word1 = "sea", word2 = "eat"

Output: 2

Explanation: You need one step to make "sea" to "ea" and another step to make "eat" to "ea".


In [3]:
def min_dist(word1, word2):
    m, n = len(word1), len(word2)
    
    dp = [[0] * (n+1) for _ in range(m+1)]
    
    for i in range(m+1):
        dp[i][0] = i
    for j in range(n+1):
        dp[0][j] = j
    
    for i in range(1, m+1):
        for j in range(1, n+1):
            if word1[i-1] == word2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(dp[i-1][j], dp[i][j-1])
    
    return dp[m][n]

word1 = "sea"
word2 = "eat"
print(min_dist(word1, word2)) 

2


####  Question 4 You need to construct a binary tree from a string consisting of parenthesis and integers. The whole input represents a binary tree. It contains an integer followed by zero, one or two pairs of parenthesis. The integer represents the root's value and a pair of parenthesis contains a child binary tree with the same structure. You always start to construct the left child node of the parent first if it exists.

Input: s = "4(2(3)(1))(6(5))"

Output: [4,2,6,3,1,5]


In [14]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right


def constructBinaryTree(s):
    def buildTree(s, index):
        if index >= len(s) or s[index] == ')':
            return None

        root_val = ''
        while index < len(s) and s[index] != '(' and s[index] != ')':
            root_val += s[index]
            index += 1

        root = TreeNode(int(root_val))

        if index < len(s) and s[index] == '(':
            index += 1
            root.left, index = buildTree(s, index)
        if index < len(s) and s[index] == '(':
            index += 1
            root.right, index = buildTree(s, index)

        if index < len(s) and s[index] == ')':
            index += 1

        return root, index

    if not s:
        return None

    root, _ = buildTree(s, 0)
    return root


def inorderTraversal(root):
    if root is None:
        return []

    result = []
    stack = []

    while root or stack:
        while root:
            stack.append(root)
            root = root.left

        root = stack.pop()
        result.append(root.val)
        root = root.right

    return result

s = "4(2(3)(1))(6(5))"
root = constructBinaryTree(s)
inorder = inorderTraversal(root)
print(inorder)

[3, 2, 1, 4, 5, 6]


#### Question 5 Given an array of characters chars, compress it using the following algorithm: Begin with an empty string s. For each group of consecutive repeating characters in chars: If the group's length is 1, append the character to s. Otherwise, append the character followed by the group's length. The compressed string s should not be returned separately, but instead, be stored in the input character array chars. Note that group lengths that are 10 or longer will be split into multiple characters in chars. After you are done modifying the input array, return the new length of the array. You must write an algorithm that uses only constant extra space.

Example 1:

Input: chars = ["a","a","b","b","c","c","c"]

Output: Return 6, and the first 6 characters of the input array should be: ["a","2","b","2","c","3"]

Explanation:

The groups are "aa", "bb", and "ccc". This compresses to "a2b2c3".


In [6]:
def compress(chars):
    r_ptr = 0
    w_ptr = 0
    cnt = 1 
    
    for i in range(1, len(chars) + 1):
        if i < len(chars) and chars[i] == chars[i - 1]:
            cnt += 1
        else:
            chars[w_ptr] = chars[r_ptr]
            w_ptr += 1
            if cnt > 1:
                cnt_str = str(cnt)
                for j in range(len(cnt_str)):
                    chars[w_ptr] = cnt_str[j]
                    w_ptr += 1
            cnt = 1
            r_ptr = i
    
    return w_ptr

chars = ["a","a","b","b","c","c","c"]
length = compress(chars)
print(length)
print(chars[:length])

6
['a', '2', 'b', '2', 'c', '3']


#### Question 6 Given two strings s and p, return an array of all the start indices of p*'s anagrams in* s. You may return the answer in any order.An Anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.

Example 1:

Input: s = "cbaebabacd", p = "abc"

Output: [0,6]

Explanation:

The substring with start index = 0 is "cba", which is an anagram of "abc".

The substring with start index = 6 is "bac", which is an anagram of "abc".


In [8]:
from collections import Counter

def find_anagrams(s, p):
    res = []
    p_freq = Counter(p) 
    s_freq = Counter(s[:len(p)])
    
    if s_freq == p_freq:
        res.append(0)
    
    for i in range(1, len(s) - len(p) + 1):
        if s_freq[s[i-1]] == 1:
            del s_freq[s[i-1]]
        else:
            s_freq[s[i-1]] -= 1
        
        if s[i + len(p) - 1] in s_freq:
            s_freq[s[i + len(p) - 1]] += 1
        else:
            s_freq[s[i + len(p) - 1]] = 1
        
        if s_freq == p_freq:
            res.append(i)
    
    return res

s = "cbaebabacd"
p = "abc"
print(find_anagrams(s, p))


[0, 6]


#### Question 7 Given an encoded string, return its decoded string. The encoding rule is: k[encoded_string], where the encoded_string inside the square brackets is being repeated exactly k times. Note that k is guaranteed to be a positive integer. You may assume that the input string is always valid; there are no extra white spaces, square brackets are well-formed, etc. Furthermore, you may assume that the original data does not contain any digits and that digits are only for those repeat numbers, k. For example, there will not be input like 3a or 2[4]. The test cases are generated so that the length of the output will never exceed 105.

Example 1:

Input: s = "3[a]2[bc]"

Output: "aaabcbc"


In [9]:
def decode_str(s):
    stk = []
    curr_cnt = 0
    curr_str = ""
    
    for char in s:
        if char.isdigit():
            curr_cnt = curr_cnt * 10 + int(char)
        elif char == '[':
            stk.append(curr_cnt) 
            stk.append(curr_str)
            curr_cnt = 0
            curr_str = ""
        elif char == ']':
            prev_str = stk.pop()
            prev_cnt = stk.pop()
            curr_str = prev_str + prev_cnt * curr_str
        else:
            curr_str += char
    
    return curr_str

s = "3[a]2[bc]"
print(decode_str(s))


aaabcbc


#### Question 8 Given two strings s and goal, return true if you can swap two letters in s so the result is equal to goal*, otherwise, return* false*.* Swapping letters is defined as taking two indices i and j (0-indexed) such that i != j and swapping the characters at s[i] and s[j]. For example, swapping at indices 0 and 2 in "abcd" results in "cbad".

Example 1:

Input: s = "ab", goal = "ba"

Output: true

Explanation: You can swap s[0] = 'a' and s[1] = 'b' to get "ba", which is equal to goal.


In [10]:
def Strings(s, goal):
    if len(s) != len(goal):
        return False

    if s == goal:
        seen = set()
        for char in s:
            if char in seen:
                return True
            seen.add(char)
        return False

    diff_pos = []
    for i in range(len(s)):
        if s[i] != goal[i]:
            diff_pos.append(i)

    if len(diff_pos) != 2:
        return False

    return s[diff_pos[0]] == goal[diff_pos[1]] and s[diff_pos[1]] == goal[diff_pos[0]]

s = "ab"
goal = "ba"
print(Strings(s, goal))


True
